In [1]:
from llm_foundation import logger
from llm_foundation.basic_structs import Provider, LMConfig
from llm_foundation.extractors import WebPageContentExtractor, CitedDocuments
from llm_foundation.lm import get_model_catalog, get_lm
from llm_foundation.routing import ToolMaster
from llm_foundation.utils import banner, show_banner
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser

from langchain.prompts import ChatPromptTemplate

2024-08-26 22:34:09 fperez-gcloud-stupid-sailor-twift root[1118223] INFO Logger root configured


Trying to configure logger root in module llm_foundation
root # of associated handlers - 0
Logging is not configured yet. Configuring it now.
Basic logging config


2024-08-26 22:34:10 fperez-gcloud-stupid-sailor-twift langchain_community.utils.user_agent[1118223] WARNING USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
@banner(text="Welcome to the Language Modeling (LLM) Foundation library", mark_fn_end=True)
def say_hi():
    logger.info("hola!")
    
say_hi()

2024-08-26 22:34:22 fperez-gcloud-stupid-sailor-twift root[1118223] INFO ************************************************************************************************************************
2024-08-26 22:34:22 fperez-gcloud-stupid-sailor-twift root[1118223] INFO *                              Welcome to the Language Modeling (LLM) Foundation library                               *
2024-08-26 22:34:22 fperez-gcloud-stupid-sailor-twift root[1118223] INFO ************************************************************************************************************************
2024-08-26 22:34:22 fperez-gcloud-stupid-sailor-twift root[1118223] INFO hola!
2024-08-26 22:34:22 fperez-gcloud-stupid-sailor-twift root[1118223] INFO ************************************************************************************************************************
2024-08-26 22:34:22 fperez-gcloud-stupid-sailor-twift root[1118223] INFO *                           End of Welcome to the Language Modeling (LLM

In [ ]:
get_model_catalog(Provider.LC, ["gpt-4o-mini"])

Provider.LC LC


2024-08-23 21:19:03 fperez-gcloud-stupid-sailor-twift httpx[67787] INFO HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"


Provider.OpenAI OpenAI


2024-08-23 21:19:03 fperez-gcloud-stupid-sailor-twift httpx[67787] INFO HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"


{'lc_gpt-4o-mini': 'gpt-4o-mini',
 'lc_gpt-4o-mini-2024-07-18': 'gpt-4o-mini-2024-07-18',
 'openai_gpt-4o-mini': 'gpt-4o-mini',
 'openai_gpt-4o-mini-2024-07-18': 'gpt-4o-mini-2024-07-18'}

In [ ]:
lm_config = LMConfig(model="gpt-4o-mini", provider=Provider.LC)
lm = get_lm(lm_config)

2024-08-23 21:19:03 fperez-gcloud-stupid-sailor-twift root[67787] INFO Creating lm object for provider LC
2024-08-23 21:19:03 fperez-gcloud-stupid-sailor-twift root[67787] INFO Creating ModelType.Chat object for model gpt-4o-mini
2024-08-23 21:19:03 fperez-gcloud-stupid-sailor-twift root[67787] INFO LM object client=<openai.resources.chat.completions.Completions object at 0x7ff077b2d640> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7ff077b64230> root_client=<openai.OpenAI object at 0x7ff077b2e6f0> root_async_client=<openai.AsyncOpenAI object at 0x7ff077e6dc70> model_name='gpt-4o-mini' temperature=0.0 openai_api_key=SecretStr('**********') openai_proxy='' max_tokens=300 created


In [ ]:
web_doc = "https://lilianweng.github.io/posts/2023-06-23-agent/"

webpage_extractor_tool = WebPageContentExtractor.build_tool(lm, CitedDocuments)

print(type(webpage_extractor_tool))
print(webpage_extractor_tool.name)
print(webpage_extractor_tool.description)
print(webpage_extractor_tool.args)

<class 'langchain_core.tools.structured.StructuredTool'>
extract
Extracts information from a list of web documents.
{'source_web_docs': {'title': 'Source Web Docs', 'anyOf': [{'type': 'string'}, {'type': 'array', 'items': {'type': 'string'}}]}}


In [ ]:
tools = [webpage_extractor_tool]
query = "calculate 12 * 4"
query_requiring_tools = f"extract cited documents from {web_doc}"

print(lm.invoke(query))

lm_with_tools = lm.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | lm_with_tools # | ToolsAgentOutputParser()
chain_response = chain.invoke({"input": query_requiring_tools})
print(type(chain_response))
print(chain_response)

chain = prompt | lm_with_tools |ToolsAgentOutputParser()
chain.invoke({"input": query_requiring_tools})

2024-08-23 21:19:04 fperez-gcloud-stupid-sailor-twift httpx[67787] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


content='12 * 4 = 48.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None} id='run-a25db894-6acf-4b74-9b40-a5139bab0b9d-0' usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21}


2024-08-23 21:19:05 fperez-gcloud-stupid-sailor-twift httpx[67787] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<class 'langchain_core.messages.ai.AIMessage'>
content='' additional_kwargs={'tool_calls': [{'id': 'call_xH4ItNZysgf2maRW9H24UHic', 'function': {'arguments': '{"source_web_docs":"https://lilianweng.github.io/posts/2023-06-23-agent/"}', 'name': 'extract'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 80, 'total_tokens': 111}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-4dfaa191-04e0-4e35-a126-6eb37f5af70b-0' tool_calls=[{'name': 'extract', 'args': {'source_web_docs': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, 'id': 'call_xH4ItNZysgf2maRW9H24UHic', 'type': 'tool_call'}] usage_metadata={'input_tokens': 80, 'output_tokens': 31, 'total_tokens': 111}


2024-08-23 21:19:05 fperez-gcloud-stupid-sailor-twift httpx[67787] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ToolAgentAction(tool='extract', tool_input={'source_web_docs': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, log="\nInvoking: `extract` with `{'source_web_docs': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tmcVQxSn1FsGpZTnGuCoTZ5y', 'function': {'arguments': '{"source_web_docs":"https://lilianweng.github.io/posts/2023-06-23-agent/"}', 'name': 'extract'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 80, 'total_tokens': 111}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ce00757d-ead9-491f-ba29-74d77fbf4c38-0', tool_calls=[{'name': 'extract', 'args': {'source_web_docs': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, 'id': 'call_tmcVQxSn1FsGpZTnGuCoTZ5y', 'type': 'tool_call'}], usage_metadata={'input_token

# Tool Master

In [ ]:
tool_master = ToolMaster(tools)

In [ ]:
new_chain = prompt | lm_with_tools | tool_master
chain_response = new_chain.invoke({"input": query_requiring_tools})
chain_response


2024-08-23 21:19:06 fperez-gcloud-stupid-sailor-twift httpx[67787] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-23 21:19:06 fperez-gcloud-stupid-sailor-twift root[67787] INFO -------------- Call Tool ---------------
2024-08-23 21:19:06 fperez-gcloud-stupid-sailor-twift root[67787] INFO Extracting <class 'llm_foundation.extractors.CitedDocuments'>
2024-08-23 21:19:06 fperez-gcloud-stupid-sailor-twift root[67787] INFO Document len: 43902
2024-08-23 21:19:06 fperez-gcloud-stupid-sailor-twift root[67787] INFO Document len without empty lines: 43620
2024-08-23 21:19:06 fperez-gcloud-stupid-sailor-twift langchain_core.callbacks.base[67787] WARNING CallbackManager.merge(): Parent run IDs do not match. Using the parent run ID of the first callback manager.
2024-08-23 21:19:06 fperez-gcloud-stupid-sailor-twift langchain_core.callbacks.base[67787] WARNING CallbackManager.merge(): Parent run IDs do not match. Using the parent run ID of the first cal

[('call_ew9J1v9SBdQMz6fodLgpfFfq',
  'extract',
  {'https://lilianweng.github.io/posts/2023-06-23-agent/': [{'title': 'Chain of thought',
     'author': 'Wei et al.',
     'year': 2022},
    {'title': 'Tree of Thoughts', 'author': 'Yao et al.', 'year': 2023},
    {'title': 'LLM+P', 'author': 'Liu et al.', 'year': 2023},
    {'author': 'Yao et al.', 'year': 2023},
    {'author': 'Shinn & Labash', 'year': 2023},
    {'author': 'Liu et al.', 'year': 2023},
    {'author': 'Liu', 'year': 2023},
    {'author': 'Laskin', 'year': 2023},
    {'author': 'Duan', 'year': 2017},
    {'author': 'Miller', 'year': 1956},
    {'author': 'Karpas et al.', 'year': 2022},
    {'author': 'Parisi et al.', 'year': 2022},
    {'author': 'Schick et al.', 'year': 2023},
    {'author': 'Shen et al.', 'year': 2023},
    {'author': 'Li et al.', 'year': 2023},
    {'author': 'Bran et al.', 'year': 2023},
    {'author': 'Boiko et al.', 'year': 2023},
    {'author': 'Park, et al.', 'year': 2023},
    {'author': 'Weng,

In [ ]:
# tool_response = tool_master.call_tools(chain_response)
# tool_response